In [1]:
import argparse
from ast import List
import copy
import datetime
import os
import shutil
import socket
import sys
from pathlib import Path
from typing import Tuple
import numpy
import torch

import decode.evaluation
import decode.neuralfitter
import decode.neuralfitter.coord_transform
import decode.neuralfitter.utils
import decode.simulation
import decode.utils
# from decode.neuralfitter.train.random_simulation import setup_random_simulation
from decode.neuralfitter.utils import log_train_val_progress
from decode.utils.checkpoint import CheckPoint


/home/lingjia/anaconda3/envs/decode_dev/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def parse_args():
    parser = argparse.ArgumentParser(description='Training Args')

    parser.add_argument('-i', '--device', default=None, 
                        help='Specify the device string (cpu, cuda, cuda:0) and overwrite param.',
                        type=str)

    parser.add_argument('-p', '--param_file', default=None,
                        help='Specify your parameter file (.yml or .json).', type=str)

    # parser.add_argument('-d', '--debug', default=False, action='store_true',
    #                     help='Debug the specified parameter file. Will reduce ds size for example.')

    parser.add_argument('-w', '--num_worker_override',default=None,
                        help='Override the number of workers for the dataloaders.',
                        type=int)

    parser.add_argument('-n', '--no_log', default=False, action='store_true',
                        help='Set no log if you do not want to log the current run.')

    # parser.add_argument('-l', '--log_folder', default=None,
    #                     help='Specify the (parent) folder you want to log to. If rel-path, relative to DECODE root.')

    parser.add_argument('-c', '--log_comment', default=None,
                        help='Add a log_comment to the run.')

    parser.add_argument('-d', '--data_path_override', default=None,
                        help='Specify your path to data', type=str)

    parser.add_argument('-is', '--img_size_override', default=None,
                        help='Override img size', type=list)


    # args = parser.parse_args()
    args, unknown = parser.parse_known_args()
    return args


In [3]:
def setup_trainer(logger, model_out, ckpt_path, device, param):
    """Set model, optimiser, loss and schedulers"""
    models_available = {
        'SigmaMUNet': decode.neuralfitter.models.SigmaMUNet,
        'DoubleMUnet': decode.neuralfitter.models.model_param.DoubleMUnet,
        'SimpleSMLMNet': decode.neuralfitter.models.model_param.SimpleSMLMNet,
    }

    model = models_available[param.HyperParameter.architecture]
    model = model.parse(param)

    model_ls = decode.utils.model_io.LoadSaveModel(model, output_file=model_out)

    model = model_ls.load_init()
    model = model.to(torch.device(device))

    # Small collection of optimisers
    optimizer_available = {
        'Adam': torch.optim.Adam,
        'AdamW': torch.optim.AdamW
    }

    optimizer = optimizer_available[param.HyperParameter.optimizer]
    optimizer = optimizer(model.parameters(), **param.HyperParameter.opt_param)

    """Loss function."""
    criterion = decode.neuralfitter.loss.GaussianMMLoss(
        xextent=param.Simulation.psf_extent[0],
        yextent=param.Simulation.psf_extent[1],
        img_shape=param.Simulation.img_size,
        device=device,
        chweight_stat=param.HyperParameter.chweight_stat)

    """Learning Rate and Simulation Scheduling"""
    lr_scheduler_available = {
        'ReduceLROnPlateau': torch.optim.lr_scheduler.ReduceLROnPlateau,
        'StepLR': torch.optim.lr_scheduler.StepLR
    }
    lr_scheduler = lr_scheduler_available[param.HyperParameter.learning_rate_scheduler]
    lr_scheduler = lr_scheduler(optimizer, **param.HyperParameter.learning_rate_scheduler_param)

    """Checkpointing"""
    checkpoint = CheckPoint(path=ckpt_path)

    """Setup gradient modification"""
    grad_mod = param.HyperParameter.grad_mod

    """Log the model (Graph) """
    try:
        dummy = torch.rand((2, param.HyperParameter.channels_in,
                            *param.Simulation.img_size), requires_grad=False).to(torch.device(device))
        logger.add_graph(model, dummy)

    except:
        print("Did not log graph.")
        # raise RuntimeError("Your dummy input is wrong. Please update it.")

    """Transform input data, compute weight mask and target data"""
    # frame_proc: x --> (x-offset)/scale
    # frame_proc = decode.neuralfitter.scale_transform.AmplitudeRescale.parse(param)
    # bg_frame_proc = None

    # if param.HyperParameter.emitter_label_photon_min is not None:
    #     # select emitters with photon > emitter_label_photon_min
    #     em_filter = decode.neuralfitter.em_filter.PhotonFilter(
    #         param.HyperParameter.emitter_label_photon_min)
    # else:
    #     em_filter = decode.neuralfitter.em_filter.NoEmitterFilter()

    # tar_frame_ix_train = (0, 0)
    # tar_frame_ix_test = (0, param.TestSet.test_size)

    """Setup Target generator consisting possibly multiple steps in a transformation sequence."""
    tar_proc = decode.neuralfitter.utils.processing.TransformSequence(
        [
            # param_tar --> phot/max, z/z_max, bg/bg_max
            decode.neuralfitter.scale_transform.ParameterListRescale(
                phot_max=param.Scaling.phot_max,
                z_max=param.Scaling.z_max,
                bg_max=param.Scaling.bg_max)
        ])

    train_IDs = numpy.arange(1,9001,1).tolist()
    val_IDs = numpy.arange(9001,10001,1).tolist()

    train_ds = decode.neuralfitter.dataset.rPSFDataset(root_dir=param.InOut.data_path,
                                                       list_IDs=train_IDs, label_path=None, 
                                                       n_max=param.HyperParameter.max_number_targets,
                                                       tar_proc=tar_proc,
                                                       img_shape=param.Simulation.img_size)

    test_ds = decode.neuralfitter.dataset.rPSFDataset(root_dir=param.InOut.data_path,
                                                       list_IDs=val_IDs, label_path=None, 
                                                       n_max=param.HyperParameter.max_number_targets,
                                                       tar_proc=tar_proc,
                                                       img_shape=param.Simulation.img_size)

    """Set up post processor"""
    if param.PostProcessing is None:
        post_processor = decode.neuralfitter.post_processing.NoPostProcessing(xy_unit='px',
                                                                              px_size=param.Camera.px_size)

    elif param.PostProcessing == 'LookUp':
        post_processor = decode.neuralfitter.utils.processing.TransformSequence([

            decode.neuralfitter.scale_transform.InverseParamListRescale(
                phot_max=param.Scaling.phot_max,
                z_max=param.Scaling.z_max,
                bg_max=param.Scaling.bg_max),

            decode.neuralfitter.coord_transform.Offset2Coordinate.parse(param),

            decode.neuralfitter.post_processing.LookUpPostProcessing(
                raw_th=param.PostProcessingParam.raw_th,
                pphotxyzbg_mapping=[0, 1, 2, 3, 4, -1],
                xy_unit='px',
                px_size=param.Camera.px_size)
        ])

    elif param.PostProcessing in ('SpatialIntegration', 'NMS'):  # NMS as legacy support
        post_processor = decode.neuralfitter.utils.processing.TransformSequence([
            # out_tar --> out_tar: photo*photo_max, z*z_max, bg*bg_max
            decode.neuralfitter.scale_transform.InverseParamListRescale(
                phot_max=param.Scaling.phot_max,
                z_max=param.Scaling.z_max,
                bg_max=param.Scaling.bg_max),
            # offset --> coordinate e.g., 0.2 --> 10.2 
            decode.neuralfitter.coord_transform.Offset2Coordinate.parse(param),

            decode.neuralfitter.post_processing.SpatialIntegration(
                raw_th=param.PostProcessingParam.raw_th, # 0.5
                xy_unit='px')
        ])

    else:
        raise NotImplementedError

    """Evaluation Specification"""
    matcher = decode.evaluation.match_emittersets.GreedyHungarianMatching.parse(param)
    # matcher = None

    return train_ds, test_ds, model, model_ls, optimizer, criterion, lr_scheduler, grad_mod, post_processor, matcher, checkpoint



In [5]:
def setup_dataloader(param, train_ds, test_ds=None):
    """Set up dataloader"""

    train_dl = torch.utils.data.DataLoader(
        dataset=train_ds,
        batch_size=param.HyperParameter.batch_size,
        drop_last=True,
        shuffle=True,
        num_workers=param.Hardware.num_worker_train,
        pin_memory=True,
        collate_fn=decode.neuralfitter.utils.dataloader_customs.smlm_collate)

    if test_ds is not None:

        test_dl = torch.utils.data.DataLoader(
            dataset=test_ds,
            batch_size=param.HyperParameter.batch_size,
            drop_last=False,
            shuffle=False,
            num_workers=param.Hardware.num_worker_train,
            pin_memory=False,
            collate_fn=decode.neuralfitter.utils.dataloader_customs.smlm_collate)
    else:

        test_dl = None

    return train_dl, test_dl

In [7]:
args = parse_args()
args.device='cuda:0'
args.param_file='/home/lingjia/Documents/rPSF/NN/param_v2.yaml'
args.data_path_override='/media/hdd/rPSF_data/rPSF/train/0620_uniformFlux'
args.img_size_override=96

In [8]:
    """Load Parameters and back them up to the network output directory"""
    param_file = Path(args.param_file)
    param = decode.utils.param_io.ParamHandling().load_params(param_file)

    # auto-set some parameters (will be stored in the backup copy)
    # param = decode.utils.param_io.autoset_scaling(param)

    # add meta information - Meta=namespace(version='0.10.0'),
    param.Meta.version = decode.utils.bookkeeping.decode_state()

    """Experiment ID"""
    if param.InOut.checkpoint_init is None:
        experiment_id = datetime.datetime.now().strftime(
            "%Y-%m-%d_%H-%M-%S") + '_' + socket.gethostname()
        from_ckpt = False
        if args.log_comment:
            experiment_id = experiment_id + '_' + args.log_comment
    else:
        from_ckpt = True
        experiment_id = Path(param.InOut.checkpoint_init).parent.name

    """Set up unique folder for experiment"""
    if not from_ckpt:
        experiment_path = Path(param.InOut.experiment_out) / Path(experiment_id)
    else:
        experiment_path = Path(param.InOut.checkpoint_init).parent

    if not experiment_path.parent.exists():
        experiment_path.parent.mkdir()

    if not from_ckpt:
        experiment_path.mkdir(exist_ok=False)

    model_out = experiment_path / Path('model.pt')
    ckpt_path = experiment_path / Path('ckpt.pt')

    # Modify parameters
    if args.num_worker_override is not None:
        param.Hardware.num_worker_train = args.num_worker_override

    """Hardware / Server stuff."""
    if args.device is not None:
        device = args.device
        # param.Hardware.device_simulation = device_overwrite  # lazy assumption
    else:
        device = param.Hardware.device

    if args.data_path_override is not None:
        param.InOut.data_path = args.data_path_override

    if args.img_size_override is not None:
        param.Simulation.img_size = [args.img_size_override,args.img_size_override]
        param.Simulation.psf_extent = [[-0.5, args.img_size_override-0.5],
                                       [-0.5, args.img_size_override-0.5], None]

        param.TestSet.frame_extent = param.Simulation.psf_extent
        param.TestSet.img_size = param.Simulation.img_size

    # Backup the parameter file under the network output path with the experiments ID
    param_backup_in = experiment_path / Path('param_run_in').with_suffix(param_file.suffix)
    shutil.copy(param_file, param_backup_in)

    param_backup = experiment_path / Path('param_run').with_suffix(param_file.suffix)
    decode.utils.param_io.ParamHandling().write_params(param_backup, param)

    if sys.platform in ('linux', 'darwin'):
        os.nice(param.Hardware.unix_niceness)
    elif param.Hardware.unix_niceness is not None:
        print(f"Cannot set niceness on platform {sys.platform}. You probably do not need to worry.")

    torch.set_num_threads(param.Hardware.torch_threads)

    """Setup Log System"""
    if args.no_log:
        logger = decode.neuralfitter.utils.logger.NoLog()

    else:
        log_folder = experiment_path

        logger = decode.neuralfitter.utils.logger.MultiLogger(
            [decode.neuralfitter.utils.logger.SummaryWriter(log_dir=log_folder,
                                                            filter_keys=["dx_red_mu", "dx_red_sig",
                                                                         "dy_red_mu",
                                                                         "dy_red_sig", "dz_red_mu",
                                                                         "dz_red_sig",
                                                                         "dphot_red_mu",
                                                                         "dphot_red_sig"]),
             decode.neuralfitter.utils.logger.DictLogger()])

    # sim_train, sim_test = setup_random_simulation(param)
    ds_train, ds_test, model, model_ls, optimizer, criterion, lr_scheduler, grad_mod, post_processor, matcher, ckpt = setup_trainer(logger, model_out, ckpt_path, device, param)
    dl_train, dl_test = setup_dataloader(param, ds_train, ds_test)

    if from_ckpt:
        ckpt = decode.utils.checkpoint.CheckPoint.load(param.InOut.checkpoint_init)
        model.load_state_dict(ckpt.model_state)
        optimizer.load_state_dict(ckpt.optimizer_state)
        lr_scheduler.load_state_dict(ckpt.lr_sched_state)
        first_epoch = ckpt.step + 1
        model = model.train()
        print(f'Resuming training from checkpoint ' + experiment_id)
    else:
        first_epoch = 0

    converges = False
    n = 0
    n_max = param.HyperParameter.auto_restart_param.num_restarts

    while not converges and n < n_max:
        n += 1

        conv_check = decode.neuralfitter.utils.progress.GMMHeuristicCheck(
            ref_epoch=1,
            emitter_avg=param.Simulation.emitter_av,
            threshold=param.HyperParameter.auto_restart_param.restart_treshold,
        )

        for i in range(first_epoch, param.HyperParameter.epochs):
            logger.add_scalar('learning/learning_rate', optimizer.param_groups[0]['lr'], i)

            if i >= 1:
                _ = decode.neuralfitter.train_val_impl.train(
                    model=model,
                    optimizer=optimizer,
                    loss=criterion,
                    dataloader=dl_train,
                    grad_rescale=param.HyperParameter.moeller_gradient_rescale,
                    grad_mod=grad_mod,
                    epoch=i,
                    device=torch.device(device),
                    logger=logger
                )

            # val_loss=avg of loss for all batches
            # test_out=list of network_output: ["loss", "x", "y_out", "y_tar", "weight", "em_tar"]
            val_loss, test_out = decode.neuralfitter.train_val_impl.test(
                model=model,
                loss=criterion,
                dataloader=dl_test,
                epoch=i,
                device=torch.device(device))

            # check when first epoch
            if not conv_check(test_out.loss[:, 0].mean(), i):
                print(f"The model will be reinitialized and retrained due to a pathological loss."
                      f"The max. allowed loss per emitter is {conv_check.threshold:.1f} vs."
                      f" {(test_out.loss[:, 0].mean() / conv_check.emitter_avg):.1f} (observed).")

                converges = False
                break

            else:
                converges = True

            # if i%10 == 0:
            """Post-Process and Evaluate"""
            log_train_val_progress.post_process_log_test(loss_cmp=test_out.loss,
                                                        loss_scalar=val_loss,
                                                        x=test_out.x, y_out=test_out.y_out,
                                                        y_tar=test_out.y_tar,
                                                        weight=test_out.weight,
                                                        em_tar=ds_test.emitter(),
                                                        px_border=-0.5, px_size=1.,
                                                        post_processor=post_processor,
                                                        matcher=matcher, logger=logger,
                                                        step=i)
            # else:
            # log_train_val_progress.log_kpi_simplified(loss_scalar=val_loss,
            #                                           loss_cmp=test_out.loss,
            #                                           logger=logger,
            #                                           step=i)


            if i >= 1:
                if isinstance(lr_scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
                    lr_scheduler.step(val_loss)
                else:
                    lr_scheduler.step()

            model_ls.save(model, None)
            if args.no_log:
                ckpt.dump(model.state_dict(), optimizer.state_dict(), lr_scheduler.state_dict(),
                          step=i)
            else:
                ckpt.dump(model.state_dict(), optimizer.state_dict(), lr_scheduler.state_dict(),
                          log=logger.logger[1].log_dict, step=i)

            """Draw new samples Samples"""
            # if param.Simulation.mode in 'acquisition':
            #     ds_train.sample(True)
            # elif param.Simulation.mode != 'samples':
            #     raise ValueError

    if converges:
        print("Training finished after reaching maximum number of epochs.")
    else:
        raise ValueError(f"Training aborted after {n_max} restarts. "
                         "You can try to reduce the learning rate by a factor of 2."
                         "\nIt is also possible that the simulated data is to challenging. "
                         "Check if your background and intensity values are correct "
                         "and possibly lower the average number of emitters.")

TypeError: 'int' object is not subscriptable

In [6]:
# sim_train, sim_test = setup_random_simulation(param)
ds_train, ds_test, model, model_ls, optimizer, criterion, lr_scheduler, grad_mod, post_processor, matcher, ckpt = setup_trainer(logger, model_out, ckpt_path, device, param)
dl_train, dl_test = setup_dataloader(param, ds_train, ds_test)

Model instantiated.
Model initialised as specified in the constructor.


In [4]:
root_dir = param.InOut.data_path
train_IDs = numpy.arange(0,9000,1).tolist()
list_IDs = train_IDs
label_path = None
label_path = label_path if label_path is not None else os.path.join(root_dir,'label.txt')



NameError: name 'param' is not defined

In [22]:
import numpy

val_IDs = numpy.arange(9001,10001,1).tolist()

label_path = '/media/hdd/rPSF_data/DECODE/simulation_figure4a_LD/label.txt'
label_raw = numpy.loadtxt(label_path)
import torch
# print(label_raw.shape[0])
# i_bol = [label_raw[ii,0] in val_IDs for ii in range(label_raw.shape[0])]
i_bol = torch.zeros(label_raw.shape[0])
for idx in val_IDs:
    i_bol += label_raw[:,0] == idx


In [18]:
import numpy

val_IDs = numpy.arange(9001,10001,1).tolist()
list_IDs = val_IDs
label_path = '/media/hdd/rPSF_data/DECODE/simulation_figure4a_LD/label.txt'

label_raw = numpy.loadtxt(label_path)
if label_raw.ndim < 2:
    label_raw = numpy.expand_dims(label_raw, axis=0)

i_bol = numpy.zeros(label_raw.shape[0],dtype=numpy.int8)
for idx in list_IDs:
    i_bol += label_raw[:,0] == idx

# print(i_bol)
label_raw_v2 = label_raw[i_bol>0,:]
labels = torch.tensor(label_raw_v2[:,[0,4,1,2,3]]).float()

In [20]:
print(i_bol)
print(label_raw[-3:,:])
print(label_raw_v2[-3:,:])


[0 0 0 ... 1 1 1]
[[ 1.000000e+04  3.921390e+01  8.870000e+00  5.139500e+00  1.687187e+02]
 [ 1.000000e+04 -2.579060e+01 -3.792250e+01  3.359400e+00  1.186925e+02]
 [ 1.000000e+04 -8.237900e+00  5.469900e+00  2.333700e+00  9.225150e+01]]
[[ 1.000000e+04  3.921390e+01  8.870000e+00  5.139500e+00  1.687187e+02]
 [ 1.000000e+04 -2.579060e+01 -3.792250e+01  3.359400e+00  1.186925e+02]
 [ 1.000000e+04 -8.237900e+00  5.469900e+00  2.333700e+00  9.225150e+01]]


In [11]:
print(label_path)
label_raw = np.loadtxt(label_path)
print(label_raw[:3,:])
print(ds_train.label_gen()[:3,:])
print(torch.max(ds_train.label_gen()[:,2]))
print(torch.max(ds_train.label_gen()[:,3]))
print(torch.max(ds_train.label_gen()[:,3]))

/media/hdd/rPSF_data/rPSF/train/0620_uniformFlux/label.txt
[[  0.      20.4191  27.5939  -4.3109 113.561 ]
 [  0.     -24.3517 -25.3649   6.2191 105.108 ]
 [  0.      -5.3202  28.1096 -13.1525 139.0421]]
tensor([[  0.0000, 113.5610,  68.4191,  75.5939,  -4.3109],
        [  0.0000, 105.1080,  23.6483,  22.6351,   6.2191],
        [  0.0000, 139.0421,  42.6798,  76.1096, -13.1525]])
tensor(81.9998)
tensor(81.9997)


In [9]:
"""Load Parameters and back them up to the network output directory"""
param_file = Path(args.param_file)
param = decode.utils.param_io.ParamHandling().load_params(param_file)

# auto-set some parameters (will be stored in the backup copy)
# param = decode.utils.param_io.autoset_scaling(param)

# add meta information - Meta=namespace(version='0.10.0'),
param.Meta.version = decode.utils.bookkeeping.decode_state()

"""Experiment ID"""
if param.InOut.checkpoint_init is None:
    experiment_id = datetime.datetime.now().strftime(
        "%Y-%m-%d_%H-%M-%S") + '_' + socket.gethostname()
    from_ckpt = False
    if args.log_comment:
        experiment_id = experiment_id + '_' + args.log_comment
else:
    from_ckpt = True
    experiment_id = Path(param.InOut.checkpoint_init).parent.name

"""Set up unique folder for experiment"""
if not from_ckpt:
    experiment_path = Path(param.InOut.experiment_out) / Path(experiment_id)
else:
    experiment_path = Path(param.InOut.checkpoint_init).parent

if not experiment_path.parent.exists():
    experiment_path.parent.mkdir()

if not from_ckpt:
    experiment_path.mkdir(exist_ok=False)

model_out = experiment_path / Path('model.pt')
ckpt_path = experiment_path / Path('ckpt.pt')

# Modify parameters
if args.num_worker_override is not None:
    param.Hardware.num_worker_train = args.num_worker_override

"""Hardware / Server stuff."""
if args.device is not None:
    device = args.device
    # param.Hardware.device_simulation = device_overwrite  # lazy assumption
else:
    device = param.Hardware.device

if args.data_path_override is not None:
    param.InOut.data_path = args.data_path_override

if args.img_size_override is not None:
    param.Simulation.img_size = [args.img_size_override,args.img_size_override]
    param.Simulation.psf_extent = [[-0.5, args.img_size_override-0.5],
                                    [-0.5, args.img_size_override-0.5], None]

    param.TestSet.frame_extent = param.Simulation.psf_extent
    param.TestSet.img_size = param.Simulation.img_size

# Backup the parameter file under the network output path with the experiments ID
param_backup_in = experiment_path / Path('param_run_in').with_suffix(param_file.suffix)
shutil.copy(param_file, param_backup_in)

param_backup = experiment_path / Path('param_run').with_suffix(param_file.suffix)
decode.utils.param_io.ParamHandling().write_params(param_backup, param)

# if torch.cuda.is_available():
#     # device_ix = 1 if device='cuda:1', = None if device='cuda'
#     _, device_ix = decode.utils.hardware._specific_device_by_str(device)
#     if device_ix is not None:
#         # do this instead of set env variable, because torch is inevitably already imported
#         torch.cuda.set_device(device)
# elif not torch.cuda.is_available():
#     device = 'cpu'

# """ Question """
# if param.Hardware.torch_multiprocessing_sharing_strategy is not None:
#     torch.multiprocessing.set_sharing_strategy(
#         param.Hardware.torch_multiprocessing_sharing_strategy)

if sys.platform in ('linux', 'darwin'):
    os.nice(param.Hardware.unix_niceness)
elif param.Hardware.unix_niceness is not None:
    print(f"Cannot set niceness on platform {sys.platform}. You probably do not need to worry.")

torch.set_num_threads(param.Hardware.torch_threads)

"""Setup Log System"""
if args.no_log:
    logger = decode.neuralfitter.utils.logger.NoLog()

else:
    log_folder = experiment_path

    logger = decode.neuralfitter.utils.logger.MultiLogger(
        [decode.neuralfitter.utils.logger.SummaryWriter(log_dir=log_folder,
                                                        filter_keys=["dx_red_mu", "dx_red_sig",
                                                                        "dy_red_mu",
                                                                        "dy_red_sig", "dz_red_mu",
                                                                        "dz_red_sig",
                                                                        "dphot_red_mu",
                                                                        "dphot_red_sig"]),
            decode.neuralfitter.utils.logger.DictLogger()])


In [ ]:
# sim_train, sim_test = setup_random_simulation(param)
ds_train, ds_test, model, model_ls, optimizer, criterion, lr_scheduler, grad_mod, post_processor, matcher, ckpt = setup_trainer(logger, model_out, ckpt_path, device, param)
dl_train, dl_test = setup_dataloader(param, ds_train, ds_test)

if from_ckpt:
    ckpt = decode.utils.checkpoint.CheckPoint.load(param.InOut.checkpoint_init)
    model.load_state_dict(ckpt.model_state)
    optimizer.load_state_dict(ckpt.optimizer_state)
    lr_scheduler.load_state_dict(ckpt.lr_sched_state)
    first_epoch = ckpt.step + 1
    model = model.train()
    print(f'Resuming training from checkpoint ' + experiment_id)
else:
    first_epoch = 0

converges = False
n = 0
n_max = param.HyperParameter.auto_restart_param.num_restarts


In [ ]:
logger.add_scalar('learning/learning_rate', optimizer.param_groups[0]['lr'], i)

_ = decode.neuralfitter.train_val_impl.train(
    model=model,
    optimizer=optimizer,
    loss=criterion,
    dataloader=dl_train,
    grad_rescale=param.HyperParameter.moeller_gradient_rescale,
    grad_mod=grad_mod,
    epoch=i,
    device=torch.device(device),
    logger=logger
)


In [ ]:
# val_loss=avg of loss for all batches
# test_out=list of network_output: ["loss", "x", "y_out", "y_tar", "weight", "em_tar"]
val_loss, test_out = decode.neuralfitter.train_val_impl.test(
    model=model,
    loss=criterion,
    dataloader=dl_test,
    epoch=i,
    device=torch.device(device))


In [ ]:
"""Post-Process and Evaluate"""
log_train_val_progress.post_process_log_test(loss_cmp=test_out.loss,
                                            loss_scalar=val_loss,
                                            x=test_out.x, y_out=test_out.y_out,
                                            y_tar=test_out.y_tar,
                                            weight=test_out.weight,
                                            em_tar=ds_test.emitter(),
                                            px_border=-0.5, px_size=1.,
                                            post_processor=post_processor,
                                            matcher=matcher, logger=logger,
                                            step=i)



if i >= 1:
    if isinstance(lr_scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
        lr_scheduler.step(val_loss)
    else:
        lr_scheduler.step()

model_ls.save(model, None)
if args.no_log:
    ckpt.dump(model.state_dict(), optimizer.state_dict(), lr_scheduler.state_dict(),
                step=i)
else:
    ckpt.dump(model.state_dict(), optimizer.state_dict(), lr_scheduler.state_dict(),
                log=logger.logger[1].log_dict, step=i)





In [7]:
print(ds_train.emitter().xyz[:3,:])
# print(ds_train.emitter().frame_ix.shape)
print(ds_train.emitter().phot[:3])
# print(ds_train.emitter().bg.shape)
# print(ds_train.emitter().get_subset_frame(1,1))
# print(ds_train.emitter().frame_ix)

# label_raw,path = ds_train.label_raw()

tensor([[68.4191, 75.5939, 15.6891],
        [23.6483, 22.6351, 26.2191],
        [42.6798, 76.1096,  6.8475]])
tensor([113.5610, 105.1080, 139.0421])


In [14]:
a = torch.tensor([[ 4.3076e-01,  5.1470e+01,  2.8865e+01,  6.1369e-01],
        [ 5.9341e-01,  5.4971e+01,  1.6139e+01,  1.0704e+00],
        [ 6.4862e-01,  4.3625e+01,  5.1123e+01, -1.0026e+00],
        [ 5.3842e-01,  4.6118e+01,  2.4503e+01, -1.3809e+00],
        [ 5.8623e-01,  7.2888e+01,  2.6713e+01,  1.6181e+00],
        [ 4.4575e-01,  5.0563e+01,  4.2720e+01, -5.3808e-01],
        [ 7.2379e-01,  4.6952e+01,  5.5178e+01,  3.2167e-01],
        [ 7.1583e-01,  3.6665e+01,  4.7480e+01, -2.1770e-02],
        [ 6.1125e-01,  5.3613e+01,  5.7800e+01,  1.2984e+00],
        [ 6.1650e-01,  6.0197e+01,  7.6087e+01,  1.2360e-02],
        [ 5.2857e-01,  5.1036e+01,  6.6435e+01,  6.8342e-01],
        [ 6.2562e-01,  7.6063e+01,  3.3609e+01, -2.1582e-01],
        [ 4.0985e-01,  6.7193e+01,  4.2250e+01, -1.7986e+00],
        [ 4.5632e-01,  6.2332e+01,  3.6541e+01,  1.1406e+00],
        [ 4.4718e-01,  2.0260e+01,  3.8910e+01,  1.9087e+00],
        [ 7.1011e-01,  7.8753e+01,  3.5229e+01, -1.4820e+00],
        [ 6.5208e-01,  2.9200e+01,  5.3470e+01, -1.8987e+00],
        [ 6.7736e-01,  1.8492e+01,  2.1681e+01,  4.5164e-01],
        [ 4.0185e-01,  1.8372e+01,  3.6160e+01, -1.8527e+00],
        [ 5.4529e-01,  4.7004e+01,  7.1986e+01,  1.5167e+00],
        [ 4.0989e-01,  1.5032e+01,  2.5290e+01,  1.6821e+00],
        [ 4.8636e-01,  5.7208e+01,  4.6808e+01,  1.2659e+00],
        [ 5.4032e-01,  5.0307e+01,  2.9100e+01,  8.7874e-01],
        [ 6.2395e-01,  5.0439e+01,  2.7731e+01,  2.4122e-01],
        [ 5.5494e-01,  5.0901e+01,  2.9253e+01,  2.4678e-01],
        [ 6.2285e-01,  5.1220e+01,  2.9807e+01,  2.5108e-01],
        [ 5.7925e-01,  4.9501e+01,  2.8861e+01,  4.6229e-01]])

In [15]:
a.shape

torch.Size([27, 4])

In [17]:
ds_train.label_gen()

torch.Size([204818, 5])


tensor([[ 0.0000e+00,  1.1356e+02,  2.0419e+01,  2.7594e+01, -4.3109e+00],
        [ 0.0000e+00,  1.0511e+02, -2.4352e+01, -2.5365e+01,  6.2191e+00],
        [ 0.0000e+00,  1.3904e+02, -5.3202e+00,  2.8110e+01, -1.3153e+01],
        ...,
        [ 9.9990e+03,  1.1099e+02,  2.9010e+00, -1.8747e+01,  2.4678e+00],
        [ 9.9990e+03,  1.2457e+02,  3.2202e+00, -1.8193e+01,  2.5108e+00],
        [ 9.9990e+03,  1.1585e+02,  1.5008e+00, -1.9139e+01,  4.6229e+00]])

In [4]:
aa = 'im1.mat'
aa.strip('im').strip('.mat')

'1'